Для каждого уникального значения в столбце Division Name 
найти топ-5 самых часто используемых слов в описании отзыва. Исключить из рассмотрения стоп-слова.

In [1]:
import nltk
from nltk.corpus import stopwords
import pandas as pd
import re
from nltk.probability import FreqDist
import numpy as np
from nltk import flatten

In [2]:
words_stops_eng = set(stopwords.words("english"))
print("Stop words: \n", words_stops_eng)

Stop words: 
 {'with', 'hers', 'be', 'hasn', 'had', 'been', 'few', 'we', 'same', 've', 'have', 'himself', 'of', 'him', "haven't", 'where', 'his', "wouldn't", "wasn't", "should've", 'their', 'now', "weren't", 'any', 'couldn', 'needn', 'mightn', "you'll", 'weren', 'whom', 'the', 'my', 'those', 'did', 'about', 'each', 'shouldn', 'wouldn', 'up', 'in', 'hadn', "aren't", 'myself', 'me', 'ours', "won't", 'yourself', 'more', 'don', 'being', 'such', "you'd", 'which', "shan't", 'because', 'she', 'before', 'haven', 'y', 'and', 'out', 'wasn', 'he', 'ourselves', 'aren', 'our', 'they', 'there', 'off', "mightn't", 'who', "shouldn't", 'them', 'having', "that'll", 'it', 'isn', 'some', 'ain', 'not', 'her', 'that', 'both', 'shan', 'themselves', 'can', "didn't", 'theirs', 'yourselves', 'but', "hasn't", 'will', 'does', 'its', 'am', 'through', 'between', 'over', "couldn't", 'above', 'then', 'these', 'is', 'to', 'ma', 'into', 'all', 'didn', 'if', 'too', 'further', "you've", 'under', 'what', "she's", 'when', 

In [3]:
wom_table = pd.read_csv("data/Womens Clothing E-Commerce Reviews.csv", sep = ",")
wom_table

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...,...
23481,23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23482,23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23483,23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
23484,23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


In [6]:
wom_table["Review Text"]

0        Absolutely wonderful - silky and sexy and comf...
1        Love this dress!  it's sooo pretty.  i happene...
2        I had such high hopes for this dress and reall...
3        I love, love, love this jumpsuit. it's fun, fl...
4        This shirt is very flattering to all due to th...
                               ...                        
23481    I was very happy to snag this dress at such a ...
23482    It reminds me of maternity clothes. soft, stre...
23483    This fit well, but the top was very see throug...
23484    I bought this dress for a wedding i have this ...
23485    This dress in a lovely platinum is feminine an...
Name: Review Text, Length: 23486, dtype: object

In [215]:
# лучший способ удаления не подходящих нам значений
w_regex = re.compile('^[a-zA-Z]*$')
a = [[word for word in nltk.word_tokenize(str(x["Review Text"])) if w_regex.search(word) != None] for _,x in wom_table.iterrows()]
a

[['Absolutely', 'wonderful', 'silky', 'and', 'sexy', 'and', 'comfortable'],
 ['Love',
  'this',
  'dress',
  'it',
  'sooo',
  'pretty',
  'i',
  'happened',
  'to',
  'find',
  'it',
  'in',
  'a',
  'store',
  'and',
  'i',
  'glad',
  'i',
  'did',
  'bc',
  'i',
  'never',
  'would',
  'have',
  'ordered',
  'it',
  'online',
  'bc',
  'it',
  'petite',
  'i',
  'bought',
  'a',
  'petite',
  'and',
  'am',
  'i',
  'love',
  'the',
  'length',
  'on',
  'hits',
  'just',
  'a',
  'little',
  'below',
  'the',
  'knee',
  'would',
  'definitely',
  'be',
  'a',
  'true',
  'midi',
  'on',
  'someone',
  'who',
  'is',
  'truly',
  'petite'],
 ['I',
  'had',
  'such',
  'high',
  'hopes',
  'for',
  'this',
  'dress',
  'and',
  'really',
  'wanted',
  'it',
  'to',
  'work',
  'for',
  'me',
  'i',
  'initially',
  'ordered',
  'the',
  'petite',
  'small',
  'my',
  'usual',
  'size',
  'but',
  'i',
  'found',
  'this',
  'to',
  'be',
  'outrageously',
  'small',
  'so',
  'smal

In [216]:
# удаляем стоп стова 
a2 = []
for x in a:
    a1 = []
    a3 = []
    for y in x:
        if y in words_stops_eng:
            a1.append(y)
        else:
            a3.append(y)
    a2.append(a3)
a2
        #if y not in words_stops_eng:
            #a1.append(y)
    #a2.append(a1)
#len(a2)

[['Absolutely', 'wonderful', 'silky', 'sexy', 'comfortable'],
 ['Love',
  'dress',
  'sooo',
  'pretty',
  'happened',
  'find',
  'store',
  'glad',
  'bc',
  'never',
  'would',
  'ordered',
  'online',
  'bc',
  'petite',
  'bought',
  'petite',
  'love',
  'length',
  'hits',
  'little',
  'knee',
  'would',
  'definitely',
  'true',
  'midi',
  'someone',
  'truly',
  'petite'],
 ['I',
  'high',
  'hopes',
  'dress',
  'really',
  'wanted',
  'work',
  'initially',
  'ordered',
  'petite',
  'small',
  'usual',
  'size',
  'found',
  'outrageously',
  'small',
  'small',
  'fact',
  'could',
  'zip',
  'reordered',
  'petite',
  'medium',
  'overall',
  'top',
  'half',
  'comfortable',
  'fit',
  'nicely',
  'bottom',
  'half',
  'tight',
  'layer',
  'several',
  'somewhat',
  'cheap',
  'net',
  'layers',
  'imo',
  'major',
  'design',
  'flaw',
  'net',
  'layer',
  'sewn',
  'directly',
  'zipper',
  'c'],
 ['I',
  'love',
  'love',
  'love',
  'jumpsuit',
  'fun',
  'flirty

In [217]:
wom_table["Review Text"] = a2 
wom_table

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,"[Absolutely, wonderful, silky, sexy, comfortable]",4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,"[Love, dress, sooo, pretty, happened, find, st...",5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,"[I, high, hopes, dress, really, wanted, work, ...",3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"[I, love, love, love, jumpsuit, fun, flirty, f...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,"[This, shirt, flattering, due, adjustable, fro...",5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...,...
23481,23481,1104,34,Great dress for many occasions,"[I, happy, snag, dress, great, price, easy, sl...",5,1,0,General Petite,Dresses,Dresses
23482,23482,862,48,Wish it was made of cotton,"[It, reminds, maternity, clothes, soft, stretc...",3,1,0,General Petite,Tops,Knits
23483,23483,1104,31,"Cute, but see through","[This, fit, well, top, see, never, would, work...",3,0,1,General Petite,Dresses,Dresses
23484,23484,1084,28,"Very cute dress, perfect for summer parties an...","[I, bought, dress, wedding, summer, cute, unfo...",3,1,2,General,Dresses,Dresses


In [218]:
#agg - Агреггация, чтобы мы могли видеть groupby
my_table = wom_table.groupby("Division Name")["Review Text"].agg(list)
my_table

Division Name
General           [[Love, dress, sooo, pretty, happened, find, s...
General Petite    [[I, love, love, love, jumpsuit, fun, flirty, ...
Initmates         [[Absolutely, wonderful, silky, sexy, comforta...
Name: Review Text, dtype: object

In [220]:
#flatten - сдлеать из двумерной одномерную(сцепить элементы)
array = []
for i in range(len(my_table)):
    t = flatten(my_table[i])
    my_dict = nltk.FreqDist(t)
    #print(my_dict.most_common(5))
    array.append(my_dict.most_common(5))
array #топ 5 по каждому Division

[[('dress', 5999), ('I', 5552), ('size', 5151), ('love', 4626), ('top', 4519)],
 [('dress', 4260), ('I', 3280), ('size', 3034), ('love', 2767), ('top', 2482)],
 [('I', 550), ('size', 504), ('love', 459), ('like', 414), ('fit', 410)]]

In [221]:
#my_table.keys()

Index(['General', 'General Petite', 'Initmates'], dtype='object', name='Division Name')

In [222]:
for i in range(0,len(my_table)):
    my_table[i] = array[i]
my_table

Division Name
General           [(dress, 5999), (I, 5552), (size, 5151), (love...
General Petite    [(dress, 4260), (I, 3280), (size, 3034), (love...
Initmates         [(I, 550), (size, 504), (love, 459), (like, 41...
Name: Review Text, dtype: object